# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [2]:
url = 'https://movies.yahoo.com.tw/'
r = requests.get(url)
soup = BeautifulSoup(r.text , 'html5lib')
search = soup.find('select' , attrs = {'name' : 'movie_id'}).find_all('option')
for i in range(1,len(search)):
    movie_name = search[i]['data-name']
    ids = search[i]['value']
    print(f'{ids} {movie_name}')

9890 葉問4：完結篇
10225 STAR WARS : 天行者的崛起
10392 復仇母親
10387 風暴過後
10380 南方車站的聚會
10373 祭旗
10370 唯你是愛
10367 屍控療法
10357 黑熊來了
10353 唐人街探案2
10351 黑計畫
10348 HUMAN LOST人間失格
10331 假面騎士劇場版ZI-O Over Quartzer
10329 巴黎小情聖
10324 每個月來訪一次的月經醬
10323 8級警戒
10297 為你存在的每一天
10291 黑色聖誕節
10257 熱氣球飛行家
10122 大說謊家
10364 特約經紀公司
10352 一屍到底番外篇：好萊塢大作戰！
10340 花生醬獵鷹的願望
10339 女孩要革命
10336 我的靈魂是愛做的
10328 絕世情歌
10287 人間失格：太宰治與他的3個女人
10279 抓狂博物館
10264 呆萌特務
10256 去年聖誕節
10134 野蠻遊戲：全面晉級
10327 夕霧花園
10254 懸案密碼前傳：瓶中信
10224 布魯克林孤兒
10044 暴走曼哈頓
10258 鋒迴路轉
10097 賽道狂人
10358 麂皮：永不滿足
10298 82年生的金智英
10271 BanG Dream！ FILM LIVE
9597 冰雪奇緣2
10302 Hello World
10281 萬萬沒想到
10226 為美好的世界獻上祝福！紅傳說
10301 山田孝之的痛苦與榮耀
10290 重甲機神：神降臨
10130 決戰中途島
10098 安眠醫生
9621 地久天長
10067 魔鬼終結者：黑暗宿命
10292 紫羅蘭永恆花園外傳－永遠與自動手記人偶－
10240 陽光普照
9992 阿達一族
10284 狙擊獵殺：救援行動
10267 盲人律師
10262 黑魔女2
10249 電影版 壞傢伙們
10205 藍波：最後一滴血
10176 屍樂園：髒比雙拼
10171 弒婚遊戲
10238 108悍將
10200 校外打怪教學
10190 信用詐欺師JP
10155 無間行動
10227 星際救援
10109 返校
10064 唐頓莊園
10199 舞孃騙很大
10191 夜鶯的哭聲
10158 花椒之味
10091 天氣之子
9894 好小男孩


### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [3]:
movie_id = 10279

In [4]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [5]:
# 指定放映地區
area_id = 28

In [6]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

十二月 19
十二月 20
十二月 21
十二月 22
十二月 23


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [35]:
# 選定要觀看的日期
date = "2019-12-19"

In [36]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼



In [37]:
soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [56]:
'''

  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
  
  Your code here.

'''

for i in html:
    print(f'電影院： {i["data-theater_name"]}')
    print(f'放映類型： {i.find_all("span" , attrs = {"class" : "tapR"})[0].string} ')
    print(i.find_all('input')[0]['value'])

電影院： 光點台北電影主題館
放映類型： 數位 
2019-12-19 11:00:00
